# Adding and Managing Memory in LangGraph

Don't have a specific tutorial to go through on this one, so I will just grab information from multiple articles and documents in langchain-ai github.

## Basic Chatbot
Need something basic in order to add memory to track.

Using Harish Neel's video series to walk through this for the first time.
YouTube video #25 of the series [here](https://www.youtube.com/watch?v=KU_FDwwL5_s)

In [7]:
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, add_messages, END
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.graph import MessagesState
from langchain.chat_models import init_chat_model

llm = init_chat_model("ollama:qwen2.5:32b", temperature=0)

In [8]:
class BasicChatState(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
def chatbot(state:BasicChatState):
    return {
        "messages": [llm.invoke(state["messages"])]
    }

In [ ]:
graph = StateGraph(BasicChatState)

graph.add_node("chatbot", chatbot)
graph.set_entry_point("chatbot")
graph.add_edge("chatbot", END)

In [ ]:
app = graph.compile()

In [ ]:
from IPython.display import Image, display

display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
input_greeting = {
    "messages": [
        {"role": "user", "content": "Hi! i am Tom"}
    ]
}

response = app.invoke(input_greeting)
print("Bot:", response["messages"][-1].content)

In [ ]:
input_question = {
    "messages": [
        {"role": "user", "content": "what's my name?"}
    ]
}

response = app.invoke(input)
print("Bot:", response["messages"][-1].content)

## Add tools

Video #26 found [here](https://www.youtube.com/watch?v=FYxvh5YQniQ)

In [9]:
from langchain_tavily import TavilySearch

search_tool = TavilySearch(max_results=2)
tools = [search_tool]

In [ ]:
search_tool.invoke("What's a node in LangGraph?")

In [ ]:
llm_with_tools = llm.bind_tools(tools=tools)

In [ ]:
def chatbot_with_tools(state:BasicChatState):
    return {
        "messages": [llm_with_tools.invoke(state["messages"])]
}

In [ ]:
def tools_router(state:BasicChatState):
    last_message = state["messages"][-1]

    # Check if last message is a tool call attribute
    if (hasattr(last_message, "tool_calls") and len(last_message.tool_calls) > 0):
        return "tool_node"
    else:
        return END

In [10]:
from langgraph.prebuilt import ToolNode

tools_node = ToolNode(tools=tools)

In [ ]:
from langgraph.prebuilt import tools_condition

graph_with_tools = StateGraph(BasicChatState)

graph_with_tools.add_node("chatbot", chatbot_with_tools)
graph_with_tools.add_node("tools_node", tools_node)
graph_with_tools.set_entry_point("chatbot")

graph_with_tools.add_conditional_edges("chatbot", tools_condition, {"tools": "tools_node", END: END})
graph_with_tools.add_edge("tools_node", "chatbot")

In [ ]:
app_with_tools = graph_with_tools.compile()

In [ ]:
display(Image(app_with_tools.get_graph().draw_mermaid_png()))

In [ ]:
input_greeting = {
    "messages": [
        {"role": "user", "content": "Hi! i am Tom"}
    ]
}

response = app_with_tools.invoke(input_greeting)
print("Bot:", response["messages"][-1].content)

In [ ]:
input_tool_question = {
    "messages": [
        {"role": "user", "content": "What's a node in LangGraph?"}
    ]
}

response = app_with_tools.invoke(input_tool_question)
print("Bot:", response["messages"][-1].content)

In [ ]:
response

## With Memory
- Harish Video #27 found [here](https://www.youtube.com/watch?v=QTaou6alCL0)
- Official documentation from LangChain-ai [here](https://langchain-ai.github.io/langgraph/tutorials/get-started/3-add-memory/)

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

app_with_memory = graph_with_tools.compile(checkpointer=checkpointer)

In [ ]:
display(Image(app_with_memory.get_graph().draw_mermaid_png()))

In [ ]:
config = {"configurable": {"thread_id": "1"}}

In [ ]:
user_input = "Hello! I'm Tom"

events = app_with_memory.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)

for event in events:
    event["messages"][-1].pretty_print()

In [ ]:
user_input = "Do you remember my name?"

events = app_with_memory.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)

for event in events:
    event["messages"][-1].pretty_print()

In [ ]:
messages = app_with_memory.get_state(config=config)

for message in messages.values["messages"]:
    message.pretty_print()

## With SqliteSaver Checkpointer
Video #28 found [here](https://www.youtube.com/watch?v=xK8g1A5Plvk)

In [ ]:
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3

conn = sqlite3.connect("checkpoints.sqlite", check_same_thread=False)
memory = SqliteSaver(conn)



In [ ]:
app_with_sqlite = graph_with_tools.compile(checkpointer=memory)

In [ ]:
config_sqlite = {"configurable": {"thread_id": "2"}}

In [ ]:
user_input = "Hello! I'm Tom"

events = app_with_sqlite.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config_sqlite,
    stream_mode="values",
)

for event in events:
    event["messages"][-1].pretty_print()

## Add Human-in-the-loop controls
- Harish videos: [29](https://www.youtube.com/watch?v=UOSMnDOC9T0) and [30](https://www.youtube.com/watch?v=9JHMLBQzU3s)
- LangChain-ai documentation: [4-human-in-the-loop](https://langchain-ai.github.io/langgraph/tutorials/get-started/4-human-in-the-loop/)

In [ ]:
from langchain_core.tools import tool
from langgraph.types import interrupt, Command

@tool 
def human_assistance(query: str) -> str:
    """Request assistance from a human."""
    human_response = interrupt({"query": query})
    return human_response["data"]

In [ ]:
tool = TavilySearch(max_results=2)
tools = [tool, human_assistance]
llm_with_tools = llm.bind_tools(tools=tools)

In [ ]:
def chatbot_with_interrupt(state:BasicChatState):
    message = llm_with_tools.invoke(state["messages"])
    # because we will be interrupting during tool execution
    # we disable parallel tool calling to avoid repeating any
    # tool invocations when we resume
    assert len(message.tool_calls) <= 1

In [ ]:
graph_builder = StateGraph(BasicChatState)

graph_builder.add_node("chatbot", chatbot_with_interrupt)
graph_builder.set_entry_point("chatbot")
tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)
graph_builder.add_conditional_edges(
    "chatbot", 
    tools_condition
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge("chatbot", END)